# 偏旁规范化以及去重

偏旁规范化，指的是让辅助码更符合 `zrm2000` 的映射规则，以此降低一个字同时对应多组辅助码的问题，从而减少重码。

`zrm2000` 的键位可参考以下链接：

1. https://www.liuchuo.net/archives/2847

2. https://zhuanlan.zhihu.com/p/122866844

3. 键位图 https://blog.csdn.net/pmo992/article/details/104963648


举个例子，我们这个方案默认的 `zrm-pinyin.dict.yaml` 里面如果存在同音多辅码的情况，比如说 `星` 字，有 `xy;ou` 和 `xy;ru` 两个码，则可以通过

1. 用 `reShape` 函数对 `zrm-pinyin.dict.yaml` 操作 `xy;ru -> xy;ou`，输出至文件 `zrm-pinyin.temp.dict.yaml`。

2. 用 `rmRepeat` 函数对 `zrm-pinyin.temp.dict.yaml` 操作干掉重复的 entry，输出至文件 `zrm-pinyin.wanted.dict.yaml`。

3. 备份原来的 `zrm-pinyin.dict.yaml`，再手动重命名 `zrm-pinyin.wanted.dict.yaml -> zrm-pinyin.dict.yaml` ，rime就会拿这个新文件当字典来用

需要下载的外部 python package: 

1. [miaoluda/hanzi_chaizi forked from howl-anderson/hanzi_chaizi](https://github.com/miaoluda/hanzi_chaizi/tree/patch-1)。请注意要下载的是 fork 版 **不要下载原版** ，否则以下必报错

导入 `sys`, `HanziChaizi` 并且熟悉后者的拆字逻辑。

In [4]:
import sys

# location of miaoluda/hanzi_chaizi package
sys.path.append("../../../hanzi_chaizi/")
from hanzi_chaizi import HanziChaizi

hc = HanziChaizi()
# 测试以下拆字结果，比如 亻 旁，输出的就是个正常的 “人”。注意这种输出的匹配，
# 不要不加测试就盲目的拿 “亻” 去下面的代码 run，那样会毫无效果
result = hc.query('伍')

print(result)

['人', '五']


In [5]:
hc.query('由')

['囗', '十']

”拆不动“的字，输出 `None`

In [8]:
print(hc.query('一'))

None


对一些极少使用的字有误伤，后果不严重。

In [27]:
print(hc.query('鯈'))
print(hc.query('擜'))

['攸', '魚']
None


In [22]:
print(hc.query('膀'))
print(hc.query('臂'))
print(hc.query('撻'))

['肉', '旁']
['肉', '辟']
['手', '達']


`reShape()` 函数用于规范化第三码（偏旁，也就是分号后面第一个辅助码），目前暂未考虑分号后面第二个辅助码，以后可能再出个 `reShape2()` 用来改进。

`rmRepeat()` 函数用于去除“完全重复的编码”。

In [25]:
def reShape(src, temp):
    # 重新map辅助码。 
    # src: "input dict" ; dst: "new file name for the output dict"
    with open(src, "r", encoding="utf-8") as f1:
        with open(temp, "w", encoding="utf-8") as f2:
            for line in f1:
                # NOTE: each line ends with a \n , (possibly) except the last one.
                if '\n' not in line:
                    line2 = line + '\n'
                if line[0].isascii() or hc.query(line[0]) is None: # '\n' belongs to ascii 
                    line2 = line
                elif ';r' in line:
                    if '彳' in hc.query(line[0])[0]:
                        # '行'字旁
                        line2 = line.replace(';r', ';x')
                    elif '人' == hc.query(line[0])[0]:
                        # '人'字旁不变，还是原来的 r。
                        # 为了防止"傝"字（人-日-etc 结构）被改！
                        line2 = line
                    elif '日' in hc.query(line[0]):
                        # '日'字旁根据规则，是 o
                        line2 = line.replace(';r', ';o')
                    else: # 剩下的 ;r 原样输出
                        line2 = line
                elif ';y' in line and '肉' in hc.query(line[0]):
                    # '月'字旁根据规则，是 o
                    line2 = line.replace(';y', ';o')
                elif ';m' in line and '目' in hc.query(line[0]):
                    # '目'字旁根据规则，是 o
                    line2 = line.replace(';m', ';o')
                elif ';t' in line and '手' in hc.query(line[0]):
                    # 提'手'旁根据规则，是 f
                    line2 = line.replace(';t', ';f')
                else: # 剩下的所有不关心的，原样输出
                    line2 = line
                f2.write(line2)

def reShape2(temp0, temp):
    pass
                
def rmRepeat(temp, dst):
    # 去重。
    # src: "input dict" ; dst: "new file name for the output dict"
    with open(temp, "r", encoding="utf-8") as f1:
        with open(dst, "w", encoding="utf-8") as f2:
            last_line = '\n'
            for line in f1:
                if line == last_line:
                    f2.write('')
                else:
                    f2.write(line)
                last_line = line


实操环节：

In [28]:
# temp.yaml 里面有“更符合规范”的码，但也有重复条目，因为 reShape() 没有删除任何一行！
reShape('../zrm_pinyin.utf8-lite.dict.yaml', 
        'temp.yaml')   

# zrm_pinyin.output.dict.yaml 里面有“更符合规范”的码，且无重复条目
rmRepeat('temp.yaml', '../zrm_pinyin.standard_utf8_lite.dict.yaml')